In [ ]:
import os

PATH = os.getcwd() + "/.cache/huggingface"
os.environ["HF_HOME"] = PATH
os.environ["HF_DATASETS_CACHE"] = PATH
os.environ["TORCH_HOME"] = PATH

import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from tqdm.notebook import tqdm
from qdrant_client import QdrantClient
from qdrant_client.models import (
    PointStruct,
    Distance,
    VectorParams,
    SparseVectorParams,
    Modifier,
)
import pandas as pd
import math
from tqdm.notebook import tqdm
from BM25 import BM25

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("../models/modernbert-embed-base-tokenizer")
model = AutoModel.from_pretrained("../models/modernbert-embed-base-model")
model = model.to(DEVICE)

bm25 = BM25(
    stopwords_dir=os.path.abspath("./stopwords"), languages=["english", "bengali"]
)

In [ ]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(
        input_mask_expanded.sum(1), min=1e-9
    )


def dense_embedding(texts: list[str]):
    encoded_queries = tokenizer(
        texts, padding=True, truncation=True, return_tensors="pt"
    )

    with torch.no_grad():
        queries_outputs = model(**encoded_queries.to(DEVICE))

    embeddings = mean_pooling(queries_outputs, encoded_queries["attention_mask"])
    embeddings = F.normalize(embeddings, p=2, dim=1)
    return embeddings.cpu().numpy()

In [ ]:
COLLECTION_NAME = "product_collection_modern_bert_base"

In [ ]:
client = QdrantClient(url="http://localhost:6333", timeout=600)
client.delete_collection(collection_name=COLLECTION_NAME)
client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config={
        "dense_vector": VectorParams(size=768, distance=Distance.COSINE)
    },
    sparse_vectors_config={"sparse_vector": SparseVectorParams(modifier=Modifier.IDF)},
)

In [ ]:
product_info_df = pd.read_csv("./final_5000_products.csv")
product_info_df.head(5)

In [ ]:
total_row = product_info_df.shape[0]
batch_size = 1
total_batch = math.ceil(total_row / batch_size)

In [ ]:
for start in tqdm(range(0, total_row, batch_size)):
    batch = product_info_df.iloc[start : start + batch_size]

    titles = batch["title"].tolist()
    descriptions = batch["description"].tolist()
    prices = batch["price"].tolist()

    texts_for_embedding = [
        f"Title:{title} | Description: {description} | Price: {price}"
        for title, description, price in zip(titles, descriptions, prices)
    ]
    dense_vectors = dense_embedding(texts_for_embedding)
    sparse_vectors = bm25.raw_embed(texts_for_embedding)

    points = []
    for idx, (batch_idx, row) in enumerate(batch.iterrows()):
        title = row["title"]
        description = row["description"]
        price = row["price"]

        points.append(
            PointStruct(
                id=batch_idx,
                vector={
                    "dense_vector": dense_vectors[idx],
                    "sparse_vector": sparse_vectors[idx],
                },
                payload={
                    "title": title,
                    "description": description,
                    "price": price,
                },
            )
        )

    operation_info = client.upsert(
        collection_name=COLLECTION_NAME, wait=True, points=points
    )
    print(operation_info, end="\r")
    # torch.cuda.empty_cache()